In [1]:
import database as db
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import clear_output
from ipywidgets import HBox, VBox
from utils import search_by_index, search_by_pred, load_data, display_all
import traceback

dm = db.DataManager('default','./to_annotate/wu.txt')


args_type = ['arg0','arg1','arg2', 'arg3','arg4','loc','tmp','prp']
arg_list = []
pred_input = widgets.Text(description='predicate',layout=Layout(width='80%'))
for arg_str in args_type:
    arg_list.append(widgets.Text(description=arg_str,layout=Layout(width='80%')))
search_pred_input = widgets.Text(description='Pred search', layout=Layout(width="80%"))
search_index_input = widgets.Text(description='Index search', layout=Layout(width="80%"))
search_button = widgets.Button(description='Search')
left_box = VBox([pred_input]+arg_list[0:3],layout=Layout(width='30%'))
right_box = VBox(arg_list[3:],layout=Layout(width='30%'))
search_box = VBox([search_pred_input, search_index_input, search_button], layout=Layout(width="30%"))
box = HBox([left_box,right_box, search_box])


def submit_result(b):
    id_ = dm.fetch_next()
    try:
        arg_dict = {}
        pred_idx = int(pred_input.value)
        pred_input.value = ''
        for idx,t in enumerate(arg_list):
            if t.value != '':
                val = t.value.split(',')
                val = [int(i) for i in val]
                if len(val) == 1:
                    val.append(val[0])
                assert val[1] >= val[0]
                arg_dict[args_type[idx]] = val
                t.value =  ''
        id_ = dm.fetch_next()
        dm.save(id_,pred_idx,arg_dict)
        print('just commit: pred:',pred_idx,'arg:',arg_dict)
    except:
        print('error! please examine the input format')
        traceback.print_exc()

def next_sentence(b):
    dm.commit()
    id_ = dm.fetch_next()
    clear_output()
    load_data(dm,id_)
    display_all(dm, [box, button_box])
    
    
def out(b):
    db.output_annotations(dm.data)
    print('output done!')

def search(b):
    pred_value = search_pred_input.value
    index_value = search_index_input.value
    if index_value != '':
        if pred_value != '':
            print('If pred and index value are both assigned, search results are based on index value.')
        search_by_index(dm, index_value)
    elif pred_value != '':
        search_by_pred(dm, pred_value)
    else:
        print('error!!! You must assign the value of pred or index.')

def previous(b):
    __id = dm.fetch_prev()
    if __id == -1:
        print('There is no previous reocrd. It is the first one.')
    else:
        dm.data.sync()
        clear_output()
        load_data(dm, __id)
        display_all(dm, [box, button_box])
        
    
submit_button = widgets.Button(description="Submit",)
next_button = widgets.Button(description='Next Sentence')
out_button = widgets.Button(description='Ouput result to json')

previous_btn = widgets.Button(description='Previous page')
jump_to = widgets.Text(description='Jump to', layout=Layout(width='30%'))
jump_to_btn = widgets.Button(description='Jump')

left_btn_box = HBox([submit_button, next_button, out_button])
right_btn_box = HBox([previous_btn, jump_to, jump_to_btn])

button_box = HBox([left_btn_box, right_btn_box])

out_button.on_click(out)
next_button.on_click(next_sentence)
submit_button.on_click(submit_result)
search_button.on_click(search)
previous_btn.on_click(previous)


id_ = dm.fetch_next()
load_data(dm,id_)
display_all(dm, [box, button_box])

possible predicate:
['有(0) ', '叫(13) ', '叫(19) ', '看(25) ', '过(26) ', '看(35) ', '过(36) ', '说(45) ', '应该(55) ', '觉得(59) ', '拍(60) ', '导演(67) ', '知道(71) ', '是(75) ', '是(78) ', '去了解(85) ']
Done labels:
pred: 有(0)	arg1: 一部电影票房3807.1万(1,4)	
pred: 叫(13)	arg0: 电影(2,2)	arg1: 什么名字(14,15)	
pred: 叫(19)	arg0: 电影(18,18)	arg1: 战神纪(20,20)	
pred: 看(25)	arg0: 你(-2)	arg1: 战神纪(20,20)	
pred: 看(35)	arg0: 我(34,34)	arg1: 战神纪(20,20)	
pred: 说(45)	arg0: 时光网短评(43,44)	arg1: 铁木真(46,46)	
pred: 应该(55)	
pred: 觉得(59)	arg0: 我(58,58)	arg1: 拍的不错(60,62)	
pred: 拍(60)	arg1: 不错(62,62)	arg2: 战神纪(20,20)	
pred: 知道(71)	arg0: 你(70,70)	arg1: 它的导演哈斯朝鲁(65,68)	
pred: 是(75)	arg0: 哈斯朝鲁(68,68)	arg1: 蒙古族人(76,76)	
pred: 是(78)	arg0: 哈斯朝鲁(68,68)	arg1: 蒙古族人(76,76)	
pred: 去了解(85)	arg0: 我(81,81)	arg1: 他是蒙古族人(74,76)	


progress:117/9232
